In [ ]:
# CRAWLER 

import requests
from bs4 import BeautifulSoup
import os
import re
import asyncio
from concurrent.futures import ThreadPoolExecutor

# cria pasta pra salvar HTMLs
os.makedirs("wikipedia_pessoas", exist_ok=True)

# base da wikipedia
URL_BASE = "https://pt.wikipedia.org"
PAGINA_INICIAL = "/"

# sets e listas pra controlar
visitados = set()  # páginas que já visitamos
fila_links = set([PAGINA_INICIAL])  # links que vamos visitar
pessoas_coletadas = 0
MAX_PESSOAS = 1000
total_visitas = 0

# cabeçalho pra parecer navegador normal
cabecalhos = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# função que limpa caracteres estranhos do nome do arquivo
def limpar_nome_arquivo(nome):
    return re.sub(r'[\\/*?:"<>|]', "", nome)

# função que verifica se a página parece de pessoa
def eh_pagina_de_pessoa(sopa):
    # procura a infobox da página
    infobox = sopa.find("table", class_=lambda c: (
        (isinstance(c, str) and "infobox" in c) or
        (isinstance(c, list) and any("infobox" in cls for cls in c))
    ))
    if infobox:
        rotulos = [th.get_text(strip=True).lower() for th in infobox.find_all("th")]
        rotulos_pessoa = ['nascimento', 'data de nascimento', 'ocupação',
                           'nome completo', 'nacionalidade', 'morte', 'cidadania']
        rotulos_nao_pessoa = ['localização', 'população', 'idioma', 'extinção', 'capital', 'território']
        if any(r in rotulos for r in rotulos_pessoa):
            if any(r in rotulos for r in rotulos_nao_pessoa):
                return False
            return True
    # se não tem infobox, olha categorias
    div_categorias = sopa.find("div", {"id": "mw-normal-catlinks"})
    if div_categorias:
        categorias = [li.get_text(strip=True).lower() for li in div_categorias.find_all("li")]
        palavras_pessoa = ['nascimentos', 'homens', 'mulheres', 'cantores', 'escritores', 'atores', 'políticos', 'biografia']
        if any(any(p in c for p in palavras_pessoa) for c in categorias):
            return True
    return False

# função que processa cada link (vai baixar página e salvar se for pessoa)
def processar_link(href):
    global pessoas_coletadas, total_visitas
    # se já visitamos ou coletamos tudo, sai
    if href in visitados or pessoas_coletadas >= MAX_PESSOAS:
        return []

    visitados.add(href)
    url = URL_BASE + href if href != "/" else URL_BASE + "/"
    total_visitas += 1
    print(f"{total_visitas} Visitando ({pessoas_coletadas}/{MAX_PESSOAS}): {url}")

    try:
        resp = requests.get(url, headers=cabecalhos, timeout=10)
        if resp.status_code != 200:
            return []
    except Exception as e:
        print(f"Erro na requisição: {e}")
        return []

    sopa = BeautifulSoup(resp.text, "html.parser")
    novos_links = []

    # se for página de pessoa, salva HTML
    if eh_pagina_de_pessoa(sopa) and pessoas_coletadas < MAX_PESSOAS:
        titulo = sopa.find("h1", id="firstHeading")
        nome_pessoa = limpar_nome_arquivo(titulo.text.strip()) if titulo else f"pessoa_desconhecida_{pessoas_coletadas}"
        nome_arquivo = f"wikipedia_pessoas/{nome_pessoa}.html"
        with open(nome_arquivo, "w", encoding="utf-8") as f:
            f.write(resp.text)
        pessoas_coletadas += 1

    # pega todos os links que parecem pessoas (primeira letra maiúscula e não contém ':')
    for a in sopa.find_all("a", href=True):
        h = a['href']
        if h.startswith("/wiki/") and ":" not in h and h[6].isupper() and h not in visitados:
            novos_links.append(h)

    return novos_links

# função assíncrona que coordena as threads
async def main():
    global fila_links
    NUM_THREADS = 12  # aumenta número de threads pra mais velocidade

    # executor que roda threads
    loop = asyncio.get_event_loop()
    with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
        while pessoas_coletadas < MAX_PESSOAS and fila_links:
            # pega um lote de links pra processar em paralelo
            lote = list(fila_links)[:NUM_THREADS]
            fila_links -= set(lote)  # remove da fila os que vamos processar
            tarefas = [loop.run_in_executor(executor, processar_link, link) for link in lote]

            # espera todos terminarem
            resultados = await asyncio.gather(*tarefas)

            # adiciona novos links que não foram visitados
            for lista in resultados:
                for link in lista:
                    if link not in visitados:
                        fila_links.add(link)

# roda o crawler
import nest_asyncio
nest_asyncio.apply()
await main()

print(f"Coletadas {pessoas_coletadas} páginas de pessoas em {total_visitas} visitas.")

1 Visitando (0/1000): https://pt.wikipedia.org/
2 Visitando (0/1000): https://pt.wikipedia.org/wiki/S%C3%A9rgio_Mendes
3 Visitando (0/1000): https://pt.wikipedia.org/wiki/Catarina,_Duquesa_de_Kent
4 Visitando (0/1000): https://pt.wikipedia.org/wiki/Giorgio_Armani
5 Visitando (0/1000): https://pt.wikipedia.org/wiki/Invas%C3%A3o_da_Ucr%C3%A2nia_pela_R%C3%BAssia_(2022%E2%80%93presente)
6 Visitando (0/1000): https://pt.wikipedia.org/wiki/Sud%C3%A3o
7 Visitando (0/1000): https://pt.wikipedia.org/wiki/Wikip%C3%A9dia
8 Visitando (0/1000): https://pt.wikipedia.org/wiki/Jacques_Charrier
9 Visitando (0/1000): https://pt.wikipedia.org/wiki/Bombardeios_navais_Aliados_no_Jap%C3%A3o_na_Segunda_Guerra_Mundial
10 Visitando (0/1000): https://pt.wikipedia.org/wiki/Arquip%C3%A9lago_japon%C3%AAs
11 Visitando (0/1000): https://pt.wikipedia.org/wiki/Elevador_da_Gl%C3%B3ria
12 Visitando (0/1000): https://pt.wikipedia.org/wiki/Jesse_James
13 Visitando (0/1000): https://pt.wikipedia.org/wiki/Funda%C3%A7%C3%A3o

In [ ]:
import os
from bs4 import BeautifulSoup
from collections import defaultdict, deque

PASTA = "wikipedia_pessoas"

# dicionários pra guardar nomes/links
nomes_para_links = {}
links_para_nomes = {}

# aqui vai ficar o grafo, cada pessoa tem um conjunto de conexões
arestas = defaultdict(set)

def normalizar_href(href):
    # tira pedaços tipo # e ? do link e deixa minúsculo
    href = href.split("#")[0]
    href = href.split("?")[0]
    return href.strip().lower()


# primeira passada: pegar nome e link canônico de cada pessoa
for arq in os.listdir(PASTA):
    if not arq.endswith(".html"):  # só entra em arquivos html
        continue

    caminho = os.path.join(PASTA, arq)
    html = open(caminho, "r", encoding="utf-8").read()
    sopa = BeautifulSoup(html, "html.parser")

    # título da página (nome da pessoa)
    titulo = sopa.find("h1", id="firstHeading")
    if not titulo:
        continue
    nome = titulo.text.strip()

    # procura o link oficial da wiki dessa pessoa
    link_a = None
    for a in sopa.select("link[rel=canonical]"):
        href = a.get("href")
        if href and "/wiki/" in href:
            link_a = href[href.find("/wiki/"):]
            break

    # salva nome/link
    if link_a:
        link_a = normalizar_href(link_a)
        nomes_para_links[nome] = link_a
        links_para_nomes[link_a] = nome


# montar as conexões do grafo
for arq in os.listdir(PASTA):
    if not arq.endswith(".html"):
        continue

    caminho = os.path.join(PASTA, arq)
    html = open(caminho, "r", encoding="utf-8").read()
    sopa = BeautifulSoup(html, "html.parser")

    titulo = sopa.find("h1", id="firstHeading")
    if not titulo:
        continue
    nome_a = titulo.text.strip()

    # pega todos os links dessa página
    for a in sopa.find_all("a", href=True):
        href = a["href"]
        # só pega links que são de wiki e não tem namespace tipo "Categoria:"
        if not href.startswith("/wiki/") or ":" in href:
            continue
        href = normalizar_href(href)

        # vê se esse link corresponde a alguém do nosso banco
        nome_b = links_para_nomes.get(href)
        if nome_b and nome_b != nome_a:
            # cria aresta nome_a -> nome_b
            arestas[nome_a].add(nome_b)


# mostra um resumo do grafo
print("Grafo construído:", len(nomes_para_links), "pessoas,", sum(len(v) for v in arestas.values()), "conexões.")


Grafo construído: 994 pessoas, 2155 conexões.


In [ ]:
# INTERFACE

# função auxiliar: tenta casar o nome digitado com o banco
def encontrar_nome(nome_digitado):
     # deixa tudo minúsculo e sem espaço no começo/fim
    nome_digitado = nome_digitado.lower().strip()
    
    # primeira tentativa: acha exato
    for nome in nomes_para_links.keys():
        if nome.lower() == nome_digitado:
            return nome
    
    # se não achou exato, tenta achar parcial (contendo a string)
    for nome in nomes_para_links.keys():
        if nome_digitado in nome.lower():
            return nome
    
    # se não achou nada, retorna None
    return None

# interface
# pede pra digitar a pessoa de origem e destino
pessoa_origem_in = input("Nome da pessoa de origem: ").strip()
pessoa_destino_in = input("Nome da pessoa de destino: ").strip()

# tenta casar os nomes digitados com os nomes que temos
pessoa_origem = encontrar_nome(pessoa_origem_in)
pessoa_destino = encontrar_nome(pessoa_destino_in)

# se não achou uma das pessoas
if not pessoa_origem or not pessoa_destino:
    print(" ❌ Não encontrei uma ou ambas as pessoas no banco coletado.")
else:
    # usa a função BFS pra achar o menor caminho (grau de separação)
    caminho = menor_caminho_bfs(arestas, pessoa_origem, pessoa_destino)
    
    # se não achou caminho
    if caminho is None:
        print(" ⚠️ Não há conexão encontrada entre as pessoas informadas.")
    else:
        # se achou, calcula grau de separação (quantos passos)
        graus = len(caminho) - 1
        print(f"\n✅ Grau de separação entre '{pessoa_origem}' e '{pessoa_destino}': {graus}")
        print("Caminho encontrado:")
        
        # mostra passo a passo
        for i, pessoa in enumerate(caminho):
            if i == 0:
                print(f"   {pessoa} (origem)")      # primeira pessoa
            elif i == len(caminho) - 1:
                print(f"   {pessoa} (destino)")    # última pessoa
            else:
                print(f"   {pessoa}")              # pessoas intermediárias



✅ Grau de separação entre 'Henrique III de Castela' e 'Henrique V de Inglaterra': 3
Caminho encontrado:
   Henrique III de Castela (origem)
   Filipe II de Espanha
   Henrique VIII de Inglaterra
   Henrique V de Inglaterra (destino)
